In [1]:
cb_coll = 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_COST_FN_VD'
sc_coll = 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_COST_FN_VD'

In [2]:
import pymongo
import numpy as np

client = pymongo.MongoClient()
db = client.metrics_causal

In [16]:
from pprint import pprint
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)

feats_pipeline = [{
    "$project": { 
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
            "stemmed":        "$parameters.stemmed",
            "num_feats":      "$parameters.num_feats_MEAN",
            "feats":          "$parameters.extractors",
            "cost_fn":        "$parameters.cost_function",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
},
{
    "$match":{
        "micro_f1_score": { "$exists" : True },
        # how many feats
        #"count": {          "$eq" :1 },
        # window width
        #"window_size": {    "$eq":13 }
    }
},
{
    "$sort":{
        #"weighted_f1_score":-1,
        "micro_f1_score": -1
        #"asof": -1
        #"count": -1
    }
},
]

In [10]:
import pandas as pd

def get_df_sorted_by_f1score(rows):
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    return df

cb_vd_df = get_df_sorted_by_f1score(cb_vd_rows)
sc_vd_df = get_df_sorted_by_f1score(sc_vd_rows)

## Coral Bleaching

In [20]:
coll = cb_coll
print(coll)
rows = [row for row in db[coll].aggregate(feats_pipeline)]
df = get_df_sorted_by_f1score(rows)
cols = "count,micro_f1_score,micro_recall,micro_precision,num_feats,cost_fn".split(",")
df[cols]

TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_COST_FN_VD


,count,micro_f1_score,micro_recall,micro_precision,num_feats,cost_fn
0,6,0.726334,0.704508,0.749556,38096.0,micro_f1_cost_plusepsilon
1,6,0.694097,0.647746,0.747592,27500.0,uniform_cost


## Skin Cancer

In [21]:
coll = sc_coll
print(coll)
rows = [row for row in db[coll].aggregate(feats_pipeline)]
df = get_df_sorted_by_f1score(rows)
cols = "count,micro_f1_score,micro_recall,micro_precision,num_feats,cost_fn".split(",")
df[cols]

TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_COST_FN_VD


,count,micro_f1_score,micro_recall,micro_precision,num_feats,cost_fn
0,4,0.78785,0.754025,0.824853,28022.0,micro_f1_cost_plusepsilon
1,4,0.78620,0.723614,0.860638,22586.0,uniform_cost


## How Many Examples on Average Per Code?

In [23]:
import dill

data_directory =  "/Users/simon.hughes/Google Drive/Phd/Data/"
root_folder = data_directory + "CoralBleaching/Thesis_Dataset/"
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays_cb = dill.load(f)

In [24]:
from load_data import load_process_essays
from window_based_tagger_config import get_config
from Settings import Settings

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
tagged_essays_test_cb = load_process_essays(**test_config)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
226 files found
226 essays processed


In [25]:
root_folder = data_directory + "SkinCancer/Thesis_Dataset/"
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(fname, "rb") as f:
    pred_tagged_essays_sc = dill.load(f)

In [26]:
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
tagged_essays_test_sc = load_process_essays(**test_config)

218 files found
218 essays processed


In [42]:
from crel_helper import get_tag_freq

tag_freq_train = get_tag_freq(pred_tagged_essays_cb, [])
crel_tags_train = list((t for t in tag_freq_train.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                )
                      )

tag_freq_test = get_tag_freq(tagged_essays_test_cb, [])
crel_tags_test = list((t for t in tag_freq_test.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                )
                     )

crels_cb = set(crel_tags_test).union(set(crel_tags_train))
len(crels_cb)

86

In [47]:
coll = "TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_TD"

for row in db[coll].find({}):
    precision, recall,ncodes = [],[],[]        
    for k in row.keys():
        #TODO - fix this to be more specific
        if k in crels_cb:
            ncodes.append(row[k]["num_codes"])
    print(np.mean(ncodes)/5)

6.990697674418605


In [40]:
from crel_helper import get_tag_freq

tag_freq_train = get_tag_freq(pred_tagged_essays_sc, [])
crel_tags_train = list((t for t in tag_freq_train.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                )
                      )

tag_freq_test = get_tag_freq(tagged_essays_test_sc, [])
crel_tags_test = list((t for t in tag_freq_test.keys() if ("->" in t) and
                not "Anaphor" in t and
                not "other" in t and
                not "rhetorical" in t and
                not "factor" in t and
                1 == 1
                )
                     )

crels_sc = set(crel_tags_test).union(set(crel_tags_train))
len(crels_sc)

49

In [48]:
coll = "TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_TD"

for row in db[coll].find({}):
    precision, recall,ncodes = [],[],[]        
    for k in row.keys():
        #TODO - fix this to be more specific
        if k in crels_sc:
            ncodes.append(row[k]["num_codes"])
    print(np.mean(ncodes)/5)

20.30204081632653
